# Python爬蟲

### 1. 進入到搜尋列表

In [8]:
from selenium import webdriver
import time
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from selenium.webdriver import ActionChains
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager

# 關閉 Alert
options = Options()
options.add_argument("--disable-notifications")

# 開啟網頁
driver = webdriver.Chrome(
    ChromeDriverManager().install(), chrome_options=options)
driver.get("https://twpat3.tipo.gov.tw/twpatc/twpatkm?@@0.6886565575531223")
driver.implicitly_wait(6)# 等待最多6秒

driver.find_element_by_xpath('//*[@id="datepicker_Start"]/input[2]').send_keys('20000101')
driver.find_element_by_xpath('//*[@id="datepicker_End"]/input[2]').send_keys('20211231')

    
driver.find_element_by_xpath('/html/body/form/table/tbody/tr[3]/td/table/tbody/tr[4]/td/span[1]/input').click()# 點選查詢
driver.implicitly_wait(10)# 等待最多10秒


record = driver.find_element_by_xpath('/html/body/form/table/tbody/tr[3]/td/table/tbody/tr[2]/td/table/tbody/tr[1]/td/div/table/tbody/tr/td[1]/select')# 選筆數格子
select = Select(record)
select.select_by_visible_text("100")# 選筆數
driver.implicitly_wait(10)# 等待最多10秒

# 選文字式顯示
driver.find_element_by_xpath('/html/body/form/table/tbody/tr[3]/td/table/tbody/tr[2]/td/table/tbody/tr[1]/td/ul[2]/li[4]/a/img').click()
driver.find_element_by_xpath('/html/body/form/table/tbody/tr[3]/td/table/tbody/tr[2]/td/table/tbody/tr[1]/td/ul[2]/li[4]/ul/li[3]/input').click()



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [C:\Users\peter\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache
<ipython-input-8-b15ab9f870c5>:16: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(


#### 問題:寫成函式呼叫?

## 2. 擷取網頁資訊

In [9]:
# 計數器歸0，重新輸入資料
count = 0

In [10]:
from collections import defaultdict
import pandas as pd

for i in range(1):# 抓多少頁
    
    list1 = []
    list2 = []
    list3 = []
    list4 = []
    list5 = []
    list6 = []
    
    driver.implicitly_wait(10)# 等待最多10秒
    
    # 抓取公開公告號號
    lication_nums = driver.find_elements_by_class_name('sumtd2012_603')
    for lication_num in lication_nums:
        list1.append(lication_num.text)

    # 抓取公開公告日
    release_dates = driver.find_elements_by_class_name('sumtd2025_603')
    for release_date in release_dates:
        list2.append(release_date.text)
    
    # 抓取申請日
    application_dates = driver.find_elements_by_class_name('sumtd2021_603')
    for application_date in application_dates:
        list3.append(application_date.text)

    # 抓取標題
    titles = driver.find_elements_by_class_name('sumtd2030_603')
    for title in titles:
        list4.append(title.text)
        
    # 抓取摘要
    summaries = driver.find_elements_by_class_name('sumtd2095_603')
    for summary in summaries:
        list5.append(summary.text)
        
    # 抓取專利範圍
    constructions = driver.find_elements_by_class_name('sumtd2096_603')
    for construction in constructions:
        list6.append(construction.text)

    # 資料轉成dict
    length = len(list1)    
    dict_from_list = {
        "lication_nums": [list1[i] for i in range(length)],
        "release_dates": [list2[i] for i in range(length)],
        "application_dates": [list3[i] for i in range(length)],
        "titles": [list4[i] for i in range(length)],
        "summaries": [list5[i] for i in range(length)],
        "constructions": [list6[i] for i in range(length)]
    }
    df = pd.DataFrame(dict_from_list)
    
    # 資料存成csv格式
    if count == 0:
        df.to_csv("oringinal_data.csv", index = False)
        count = count + 1
    else:
        df.to_csv("oringinal_data.csv", mode = 'a', header = False, index = False)

    
    driver.find_element_by_name('_IMG_次頁').click()# 點選下一頁

# 關driver
driver.quit()

#### 問題:需解決.csv亂碼?字典排列形式有更好的?設計避免重複抓取的判斷?儲存在更好的SQL?

## 3.資料匯入程式

In [11]:
# csv輸入至oringinal_csv 
df = pd.read_csv("oringinal_data.csv", encoding="utf-8-sig")
df.head(3)

,lication_nums,release_dates,application_dates,titles,summaries,constructions
0,M620346,2021/11/21,2021/09/29,川流式水力發電系統,本創作係揭露一種川流式水力發電系統，其係建構於一外部河川之沿岸上，並包括有彼此依序相連通之一...,1.一種川流式水力發電系統，包括： 一堤堰，係設於一外部河川之沿岸； 一渦流部，係設於該堤堰...
1,M620327,2021/11/21,2021/09/06,撐開螺絲,本創作係一種撐開螺絲，撐開螺絲包含有一錐頭螺栓、一擴張環、一填縫套筒及一固定螺帽。錐頭螺栓具...,1.一種撐開螺絲，其用以將一連接件固定於一被固定物上，該連接件具有一止轉孔，該被固定物具有一...
2,M620322,2021/11/21,2021/09/03,具食物保鮮致冷裝置之烹煮台,一種具食物保鮮致冷裝置之烹煮台，主要係包含食物保鮮致冷裝置，以及結合於該食物保鮮致冷裝置之烹...,1.一種具食物保鮮致冷裝置之烹煮台，包含： 一食物保鮮致冷裝置，係設有一承架，於該承架的上方...


## 4.中文分詞

In [12]:
import re
from bs4 import BeautifulSoup

# 去除\n
def clean_post_cont(x):
    soup = BeautifulSoup(x, 'html.parser')
    result = soup.get_text()
    return re.sub(r"\r|\n|\t", "", result)

df["titles"] = df["titles"].map(clean_post_cont)
df["summaries"] = df["summaries"].map(clean_post_cont)
df["constructions"] = df["constructions"].map(clean_post_cont)
df.head(3)

,lication_nums,release_dates,application_dates,titles,summaries,constructions
0,M620346,2021/11/21,2021/09/29,川流式水力發電系統,本創作係揭露一種川流式水力發電系統，其係建構於一外部河川之沿岸上，並包括有彼此依序相連通之一...,1.一種川流式水力發電系統，包括： 一堤堰，係設於一外部河川之沿岸； 一渦流部，係設於該堤堰...
1,M620327,2021/11/21,2021/09/06,撐開螺絲,本創作係一種撐開螺絲，撐開螺絲包含有一錐頭螺栓、一擴張環、一填縫套筒及一固定螺帽。錐頭螺栓具...,1.一種撐開螺絲，其用以將一連接件固定於一被固定物上，該連接件具有一止轉孔，該被固定物具有一...
2,M620322,2021/11/21,2021/09/03,具食物保鮮致冷裝置之烹煮台,一種具食物保鮮致冷裝置之烹煮台，主要係包含食物保鮮致冷裝置，以及結合於該食物保鮮致冷裝置之烹...,1.一種具食物保鮮致冷裝置之烹煮台，包含： 一食物保鮮致冷裝置，係設有一承架，於該承架的上方...


#### 問題:需去除html.parser以加速程式運行?

In [13]:
import jieba

def do_cuts_words(param_df):
    sentence = param_df["constructions"]
    words = list(jieba.cut(sentence))
    result = []
    for word in words:
        if not word or len(word) == 0 or len(word) == 1:
            continue
        word = word.lower()# 轉小寫
        result.append(word)
    return " ".join(result)

df["words"] = df.apply(do_cuts_words, axis = 1)
# 儲存至csv
df[["lication_nums", "release_dates", "application_dates", "titles", "summaries", "words"]].to_csv("wordsegs_data.csv", index = False)

df.head(5)

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\peter\AppData\Local\Temp\jieba.cache
Loading model cost 0.656 seconds.
Prefix dict has been built successfully.


,lication_nums,release_dates,application_dates,titles,summaries,constructions,words
0,M620346,2021/11/21,2021/09/29,川流式水力發電系統,本創作係揭露一種川流式水力發電系統，其係建構於一外部河川之沿岸上，並包括有彼此依序相連通之一...,1.一種川流式水力發電系統，包括： 一堤堰，係設於一外部河川之沿岸； 一渦流部，係設於該堤堰...,一種 川流 水力 發電系統 包括 堤堰 外部 河川 沿岸 一渦 流部 堤堰 上且 相距 外部...
1,M620327,2021/11/21,2021/09/06,撐開螺絲,本創作係一種撐開螺絲，撐開螺絲包含有一錐頭螺栓、一擴張環、一填縫套筒及一固定螺帽。錐頭螺栓具...,1.一種撐開螺絲，其用以將一連接件固定於一被固定物上，該連接件具有一止轉孔，該被固定物具有一...,一種 開螺絲 用以 將一連 接件 固定 固定物 該連 接件 具有 一止 轉孔 固定物 具有 ...
2,M620322,2021/11/21,2021/09/03,具食物保鮮致冷裝置之烹煮台,一種具食物保鮮致冷裝置之烹煮台，主要係包含食物保鮮致冷裝置，以及結合於該食物保鮮致冷裝置之烹...,1.一種具食物保鮮致冷裝置之烹煮台，包含： 一食物保鮮致冷裝置，係設有一承架，於該承架的上方...,一種 食物 保鮮 致冷 裝置 烹煮 包含 食物 保鮮 致冷 裝置 設有 一承架 承架 上方 ...
3,M620321,2021/11/21,2021/09/03,餐飲製作設備,一種餐飲製作設備，主要包括有一點餐部、一調理部、一食材預煮部、一食材儲存部、一廚具清洗部及一...,1.一種餐飲製作設備，係設有一容置空間，於該容置空間依序至少設有一點餐部、一調理部、一食材儲...,一種 設有 一容置 空間 容置 空間 依序 至少 設有 一點 餐部 一調 理部 一食材 儲存...
4,M620318,2021/11/21,2021/09/02,應用程式異常偵測系統,本創作為一種應用程式異常偵測系統，包括根據至少一應用程式之日誌資料，蒐集特徵參數，並分別輸入...,1.一種應用程式異常偵測系統，包括： 一特徵參數蒐集裝置，根據至少一應用程式之日誌資料，蒐集...,一種 應用 程式 異常 偵測 系統 包括 一特 徵參數 蒐集 裝置 根據 至少 一應用 程式...
